# Notebook Dashboard

In [2]:
from PIL.Image import new, Image, open
from random import SystemRandom
from HardenedSteel.objects import CounterObject
from IPython.display import display
from PIL import ImageDraw, ImageFont
from os import walk
from os.path import join
from time import time_ns

In [3]:
retrieve_counter: CounterObject = CounterObject()

width: int = 512
height: int = 512

background: Image = new(
    mode='RGB', 
    size=(width, height), 
    color='white'
)

In [4]:
def generate_new_random_background() -> Image:
    global background, width, height

    background = new(
        mode='RGB', 
        size=(width, height), 
        color='white'
    )
    
    generator: SystemRandom = SystemRandom()

    pixels = background.load()

    for x in range(width):
        for y in range(height):
            red, green, blue = pixels[x, y]

            red: int = generator.randint(0, 255)
            green: int = generator.randint(0, 255)
            blue: int = generator.randint(0, 255)

            pixels[x, y] = (
                red, 
                green, 
                blue
            )

    return background

generate_new_random_background()

def get_background() -> Image:
    global background, retrieve_counter
    retrieve_counter.increment()

    if int(retrieve_counter) == 1:
        retrieve_counter.reset()
        generate_new_random_background()

    return background

def generate_canvas() -> Image:
    return get_background().copy()

In [5]:
fonts_found: list = list()

def is_font(filename: str) -> bool:
    splitted_at = filename.split('.')
    last_index = len(splitted_at) - 1
    return splitted_at[last_index] == 'ttf'

In [6]:
if fonts_found is None:
    fonts_found = list()

if len(fonts_found) == 0:
    for root, directories, files in walk(
        '/mnt/c/Windows/Fonts', 
        topdown=False
    ):
        for file in files:
            full_path: str = join(root, file)

            if is_font(file):
                fonts_found.append(full_path)

In [7]:
def select_random_font() -> str:
    global fonts_found
    generator: SystemRandom = SystemRandom()
    last_index: int = len(fonts_found) - 1
    return fonts_found[generator.randint(0, last_index)]

In [24]:
number_of_batches: int = 1500
text: str = 'F'
save_at: str = '/tmp/output/'

width_margin: int = 125
height_margin: int = 125

In [25]:
selected_image: Image | None = None
font_max_size: int = height + 256

for i in range(number_of_batches):
    save_as: str = join(save_at, str(str(time_ns()) + '.jpg'))
    generator: SystemRandom = SystemRandom()

    font_size: int = generator.randint(
        32, 
        font_max_size
    )

    selected_image: Image = generate_canvas()
    font = ImageFont.truetype(
        select_random_font(), 
        font_size, 
        encoding='UTF-8'
    )

    context = ImageDraw.Draw(selected_image)
    
    position_x: int = generator.randint(
        (64 - width_margin), 
        (256 + width_margin)
    )
    
    position_y: int = generator.randint(
        (64 - height_margin), 
        (256 + height_margin)
    )


    font_color: tuple = (
        generator.randint(0, 255), 
        generator.randint(0, 255), 
        generator.randint(0, 255)
    )

    context.text(
        (position_x, position_y), 
        text, 
        font=font, 
        fill=font_color,
        align='center',
        spacing=0
    )

    save_quality: int = generator.randint(1, 100)
    
    selected_image.save(
        save_as,
        subsamling=generator.randint(0, 1),
        quality=save_quality
    )